This notebook can be used to run the simulation instead of having to type the commands into the command line interface

In [1]:
# Copyright (c) 2021 *****************. All rights reserved.
# Licensed under the Apache License, Version 2.0, which is in the LICENSE file.

import multiprocessing as mp
import os
import time
from pathlib import Path
import generate_input_files

# Specify the location of your local MODEL_RUNS directory

In [2]:

# specify the location of the model runs folder you will be using, relative to the switch_model directory
# the default location is the MODEL_RUNS directory within the Git repo
# however, if you want to save model runs in a alternate location, such as a Box folder, you could use something like:
#model_runs_folder = '../../../Box/Supply/24x7 Time-Coincident Work/MODEL_RUNS'
model_runs_folder = '../MODEL_RUNS'

# enter the name of the folder within your MODEL_RUNS folder where the input and output files for this specific model run are located
model_run_name = 'generic_office_example'

model_workspace = Path.cwd() / f'{model_runs_folder}/{model_run_name}'

# Generate Model Inputs

In [ ]:
# Enter the Timezone for your analysis
timezone = 'US/Pacific'

generate_input_files.generate_inputs(model_workspace, timezone)

# Solve Model Scenarios

### 1. Get the number of CPU cores (processors) that your computer has
If you want to solve multiple scenarios in parallel, you should only run n-1 solvers, where n is the number of CPU cores on your machine.

In [3]:
num_processors = mp.cpu_count()
print(f'Max # of parallel threads you can run: {num_processors-1}')

Max # of parallel threads you can run: 3


The following cell will actually run the scenarios. Each time you run the following cell, it will open a new parallel solver window.

In [5]:
os.system(f'start cmd /k "cd {model_runs_folder}\{model_run_name} & activate switch_247 & switch solve-scenarios"') 

0